# Demo: Generate Flood Map from Sentinel 1/2 Data

This demo classifies two dates in SE Kansas during extreme flood event in May 2019 https://www.weather.gov/ict/event_20190520

MNDWI (Xu, 2006) band calculation is used to classify flooding.
Xu, H. (2006). Modification of normalised difference water index (NDWI) to enhance open water features in remotely sensed imagery. International Journal of Remote Sensing, 27(14), 3025–3033. https://doi.org/10.1080/01431160600589179

Pre-requiste: Download imagery from Coperniucus Sentinel 1 or 2 in preferrred manner. Save to data /examples/data. 20 meter Bands 3 (green) and 11 (SWIR1) are required. Optionally, include 20 meter cloud mask.

Examples used stored on s3://fim-services-data/f1/flood_sentinel/

Data Source: https://dataspace.copernicus.eu/data-collections 

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import rasterio
from matplotlib.patches import Patch
from numpy.typing import NDArray

## Functions

In [ ]:
def prepare_inputs(
    green: Path, swir1: Path, cloud_mask: Path | None = None
) -> tuple[NDArray, NDArray, NDArray, dict]:
    """Read green, SWIR, and optionally cloud mask from inputs to array

    Inputs will be checked for spatial alignment.

    Parameters
    ----------
    green : Path
        Green band file path - must be same scene as SWIR1
    swir1 : Path
        SWIR1 band file path - must be same scene as Green
    cloud_mask : Path | None, optional
        Cloud mask file path from same scene,, by default None

    Returns
    -------
    tuple[NDArray,NDArray, NDArray, dict]
        Arrays in order of: green, SWIR1, cloud, rasterio profile
    """
    with rasterio.open(green, mode="r") as src:
        prof = src.profile
        tr_g = prof["transform"]
        arr_green = src.read(1).astype(np.float32)

    with rasterio.open(swir1, mode="r") as src:
        arr_swir1 = src.read(1).astype(np.float32)
        tr_s = src.profile["transform"]

    if cloud_mask:
        with rasterio.open(cloud_mask, mode="r") as src:
            arr_cloud = src.read(1)
    else:
        arr_cloud = None

    assert arr_green.shape == arr_swir1.shape, (
        "Input files are not same shape - provide files from same scene"
    )
    assert tr_g == tr_s, "Input files do not have same bounds - provide files from same scene"

    return arr_green, arr_swir1, arr_cloud, prof


def calculate_mnwdi_flood(
    green: NDArray,
    swir1: NDArray,
    flood_threshold: float = 0.09,
    cloud_mask: NDArray | None = None,
    cloud_threshold: int = 50,
    input_nodata: int | float = 0,
    output_nodata: int | float = -999,
) -> NDArray:
    """Calculate MNDWI (Modified Normalized Water Index) from Sentinel 1/2 data

    MNWDI uses green and SWIR1 bands. Optionally set a cloud mask.
    Sentinel uses 0 for no data. 0 is a valid output so set output to new no data.

    MNDWI = (green - SWIR1) / (green + SWIR1)

    Parameters
    ----------
    green : NDArray
        Green band array - must be same scene as SWIR1
    swir1 : NDArray
        SWIR1 band array - must be same scene as Green
    flood_threshold : float, optional
        MNNWDI flood threshold value. If greater than threshold, pixel is flooded.
        0.09 is taken from original paper, by default 0.09
    cloud_mask : NDArray | None, optional
        Cloud mask file path from same scene, by default None
    cloud_threshold : int, optional
        Threshold to consider pixel cloud. If greater than threshold, pixel is removed, by default 50
    input_nodata : int | float, optional
        NoData value in inputs. In sentinel, this is 0 and is not set in raster profile, by default 0
    output_nodata : int | float, optional
        NoData value for output. Will be set in raster profile, by default -999

    Returns
    -------
    ArrayLike
        MNWDI array
    """
    # ensure float division
    green = green.astype(np.float32)
    swir1 = swir1.astype(np.float32)

    # calcule MNWDI: (green - SWIR1) / (green + SWIR1)
    with np.errstate(divide="ignore", invalid="ignore"):
        arr_out = np.true_divide((green - swir1), (green + swir1))
        arr_out[arr_out == np.inf] = 0

    # cloud mask if desired
    if cloud_mask is not None:
        arr_out = np.where(cloud_mask > cloud_threshold, np.nan, arr_out)

    # mask nodata value
    arr_out = np.where(arr_out == input_nodata, np.nan, arr_out)

    # save off MNWDI array
    mndwi = arr_out.copy()

    # classify and handle null
    arr_out[arr_out >= flood_threshold] = 1
    arr_out[np.isnan(arr_out)] = output_nodata
    arr_out[(arr_out != output_nodata) & (arr_out < flood_threshold)] = 0

    return mndwi, arr_out


def generate_flood(
    out_path: Path,
    out_mndwi_path: Path,
    green: Path,
    swir1: Path,
    cloud_mask: Path | None = None,
    input_nodata: int | float = 0,
    output_nodata: float | int = -999,
    flood_threshold: float = 0.09,
    cloud_threshold: int = 50,
):
    """Generate a flood extent from Sentinel 1/2 data using MNDWI (Modified Normalized Water Index)

    Reads from sources, calculates, MNWDI, and saves output
    MNWDI uses green and SWIR1 bands. Optionally set a cloud mask.
    Sentinel uses 0 for no data. 0 is a valid output so set output to new no data.

    MNDWI = (green - SWIR1) / (green + SWIR1)

    Parameters
    ----------
    out_path : Path
        output file path
    out_mndwi_path : Path
        path for intermediate MNDWI output
    green : Path
        Green band file path - must be same scene as SWIR1
    swir1 : Path
        SWIR1 band file path - must be same scene as Green
    cloud_mask : Path | None, optional
        Cloud mask file path from same scene, by default None
    input_nodata : int | float, optional
        NoData value in inputs. In sentinel, this is 0 and is not set in raster profile, by default 0
    output_nodata : float | int, optional
        NoData value for output. Will be set in raster profile, by default -999
    flood_threshold : float, optional
        MNNWDI flood threshold value. If greater than threshold, pixel is flooded.
        0.09 is taken from original paper, by default 0.09
    cloud_threshold : int, optional
        Threshold to consider pixel cloud. If greater than threshold, pixel is removed, by default 50
    """
    green, swir1, cloud_mask, prof = prepare_inputs(green=green, swir1=swir1, cloud_mask=cloud_mask)
    mndwi, flood = calculate_mnwdi_flood(
        green=green,
        swir1=swir1,
        cloud_mask=cloud_mask,
        input_nodata=input_nodata,
        output_nodata=output_nodata,
        flood_threshold=flood_threshold,
        cloud_threshold=cloud_threshold,
    )

    prof.update(dtype=np.float32, compression="deflate", tiled="YES", driver="GTiff", nodata=output_nodata)
    with rasterio.open(out_path, "w", **prof) as dst:
        dst.write(flood, 1)

    with rasterio.open(out_mndwi_path, "w", **prof) as dst:
        dst.write(mndwi, 1)

    del flood, mndwi, green, swir1, cloud_mask


def plot_flood(flood: NDArray, nodata: int | float, title: str):
    """Simple plot of flood output

    Parameters
    ----------
    flood : NDArray
        Binary flood array
    mndwi : NDArray
        MNDWI array
    nodata : int | float
        No data value
    title : str
        Title for plot
    """
    f, ax = plt.subplots()
    flood[flood == nodata] = np.nan
    cmap = plt.get_cmap("viridis")
    rgba = [cmap(0), cmap(0.99)]
    legend_elements = [
        Patch(facecolor=rgba[1], edgecolor="black", label="Flooded"),
        Patch(facecolor=rgba[0], edgecolor="black", label="Non-flooded"),
    ]

    ax.imshow(flood, cmap=cmap)
    ax.legend(handles=legend_elements, loc="upper left")
    ax.set_axis_off()
    ax.set_title(f"Flood {title}")

## Examples

In [ ]:
# 5/30/2019
wd = Path("./data/prototype")
b_3 = wd / "2019_05_30/inputs/T15STB_20190530T170851_B03_20m.jp2"
b_11 = wd / "2019_05_30/inputs/T15STB_20190530T170851_B11_20m.jp2"
cloud = wd / "2019_05_30/inputs/MSK_CLDPRB_20m.jp2"
out_path = wd / "2019_05_30/outputs/flood_20190530.tif"
out_mndwi = wd / "2019_05_30/outputs/mndwi_20190530.tif"

generate_flood(out_path=out_path, green=b_3, swir1=b_11, cloud_mask=cloud, out_mndwi_path=out_mndwi)

with rasterio.open(out_path, "r") as src:
    flood = src.read(1)

plot_flood(flood, nodata=-999, title="2019-05-30")

In [ ]:
# 5/27/2019
wd = Path("./data/prototype")
b_3 = wd / "2019_05_27/inputs/T15STB_20190527T165901_B03_20m.jp2"
b_11 = wd / "2019_05_27/inputs/T15STB_20190527T165901_B11_20m.jp2"
cloud = wd / "2019_05_27/inputs/MSK_CLDPRB_20m.jp2"
out_path = wd / "2019_05_27/output/flood_20190527.tif"
out_mndwi = wd / "2019_05_27/output/mndwi_20190527.tif"

generate_flood(out_path=out_path, green=b_3, swir1=b_11, cloud_mask=cloud, out_mndwi_path=out_mndwi)

with rasterio.open(out_path, "r") as src:
    flood = src.read(1)

plot_flood(flood, nodata=-999, title="2019-05-27")

In [ ]:
# 5/27/2019 - no cloud mask
wd = Path("./data/prototype")
b_3 = wd / "2019_05_27/inputs/T15STB_20190527T165901_B03_20m.jp2"
b_11 = wd / "2019_05_27/inputs/T15STB_20190527T165901_B11_20m.jp2"
out_path = wd / "2019_05_27/output/flood_20190527_no_cloud.tif"
out_mndwi = wd / "2019_05_27/output/mndwi_20190527_no_cloud.tif"

generate_flood(out_path=out_path, green=b_3, swir1=b_11, out_mndwi_path=out_mndwi)

with rasterio.open(out_path, "r") as src:
    flood = src.read(1)

plot_flood(flood, nodata=-999, title="2019-05-27 - No cloud mask")